In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import loguniform
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
import sys
sys.path.append("../src")
from myproject.pipelines.preprocessing_pipeline import preprocessing_pipeline
from myproject.pipelines.clean_pipeline import cleaning_pipeline
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
    ConfusionMatrixDisplay
)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
import optuna
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("credit_classification_ml")

<Experiment: artifact_location='mlflow-artifacts:/454137213308002966', creation_time=1764624674955, experiment_id='454137213308002966', last_update_time=1764624674955, lifecycle_stage='active', name='credit_classification_ml', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [20]:
df = pd.read_csv("/home/jakub/Pulpit/Credit Classification ML/notebooks/no_important/plik.csv")

le = LabelEncoder()
df["Credit_Score_encoded"] = le.fit_transform(df["Credit_Score"])
class_mapping = dict(enumerate(le.classes_))
print("class mapping:", class_mapping)

class mapping: {0: 'Good', 1: 'Poor', 2: 'Standard'}


In [21]:
X = df.drop(columns=["Credit_Score"])
y = df["Credit_Score_encoded"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1_macro": "f1_macro",
    "precision_macro": "precision_macro",
    "recall_macro": "recall_macro",
}


In [22]:
cat_clf = Pipeline([
    ("cat_clf", CatBoostClassifier(
        loss_function="MultiClass",
        eval_metric="TotalF1",     # wewnętrzna metryka; do porównania użyjemy sklearn scoring
        depth=6,                   # sensowny baseline
        learning_rate=0.1,
        iterations=1000,
        l2_leaf_reg=3.0,
        random_state=42,
        thread_count=4,
        verbose=0,
        allow_writing_files=False,
    ))
])


In [23]:
cv_result_cat = cross_validate(
    cat_clf,
    X_train,
    y_train,
    scoring=scoring,
    cv=cv,
    n_jobs=-1,    # ważne przy CatBoost
    verbose=1,
)

cv_cat_mean = {
    f"cv_{metric}_mean_cat": float(cv_result_cat[f"test_{metric}"].mean())
    for metric in scoring.keys()
}

cv_cat_std = {
    f"cv_{metric}_std_cat": float(cv_result_cat[f"test_{metric}"].std())
    for metric in scoring.keys()
}

import pandas as pd
cv_cat_metrics = {**cv_cat_mean, **cv_cat_std}
pd.DataFrame([cv_cat_metrics])


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "_catboost.pyx", line 2534, in _catboost.get_float_feature
  File "_catboost.pyx", line 1228, in _catboost._FloatOrNan
  File "_catboost.pyx", line 1023, in _catboost._FloatOrNanFromString
TypeError: Cannot convert 'May' to float

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimato

In [ ]:
cat_clf.fit(X_train, y_train)

y_pred_cat = cat_clf.predict(X_test)

test_metrics_cat = {
    "test_accuracy_cat": accuracy_score(y_test, y_pred_cat),
    "test_balanced_accuracy_cat": balanced_accuracy_score(y_test, y_pred_cat),
    "test_f1_macro_cat": f1_score(y_test, y_pred_cat, average="macro"),
    "test_precision_macro_cat": precision_score(y_test, y_pred_cat, average="macro"),
    "test_recall_macro_cat": recall_score(y_test, y_pred_cat, average="macro"),
}

pd.DataFrame([test_metrics_cat])

/home/jakub/Pulpit/Credit Classification ML/notebooks/../src/myproject/features/transformers.py:213: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  parts = parts.fillna(0)
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['Credit_History_Age']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/home/jakub/Pulpit/Credit Classification ML/notebooks/../src/myproject/features/transformers.py:213: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_down

,test_accuracy_cat,test_balanced_accuracy_cat,test_f1_macro_cat,test_precision_macro_cat,test_recall_macro_cat
0,0.74615,0.726007,0.727308,0.729123,0.726007
